<a href="https://colab.research.google.com/github/Prajna1999/eng-or-mt/blob/master/tokenizer_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece

In [ ]:
!pip install --upgrade huggingface_hub


In [2]:
from huggingface_hub import login
login()

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_fast=False)
print(tokenizer.bos_token)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

<s>


In [40]:
print(tokenizer.total_vocab_size)

32768


In [142]:
import unicodedata
import regex
class TokenizerWrapper:
    def __init__(
        self,
        base_tokenizer,
        suffixes=None,
        named_entities=None,
        compound_roots=None,
        force_suffix_split=False,
        enable_fuzzy_split=False
        ):
      self.tokenizer=base_tokenizer
      self.vocab=self.tokenizer.get_vocab()

      # known Odia suffixes. In future expand the list
      self.suffixes=suffixes or[ "ମାନେ", "ଗୁଡ଼ିକ", "ରେ", "କୁ", "ରୁ", "ଟି", "ଙ୍କ","ଙ୍କୁ"]
      self.named_entities=named_entities or ["ଜଗନ୍ନାଥ", "ଭୁବନେଶ୍ୱର", "କଟକ", "ପୁରୀ", "ବାଲେଶ୍ୱର", "ସମ୍ବଲପୁର",
    "ଓଡ଼ିଶା", "ଝାରସୁଗୁଡ଼ା", "ଗଞ୍ଜାମ", "ମୟୂରଭଞ୍ଜ", "ନୟାଗଡ଼", "କଳାହାଣ୍ଡି",
    "ଖୋର୍ଦ୍ଧା", "ନୂଆପଡା", "ନବରଙ୍ଗପୁର", "ସୁନ୍ଦରଗଡ଼", "ବରଗଡ଼", "କନ୍ଧମାଳ",
    "ଗଞ୍ଜେଶ୍ୱର", "ଗୋପାଳପୁର", "ପାରାଦୀପ", "ଅନୁଗୁଳ", "ଦେବଗଡ଼", "କେନ୍ଦୁଝର",
    "ବୌଦ୍ଧ", "ମଲକାନଗିରି", "ଗଣେଶ", "ଶିବ", "ଲିଙ୍ଗରାଜ", "ମାଓ", "ମହାନଦୀ",
    "ଚିଲିକା", "ରଥଯାତ୍ରା", "କୋଣାର୍କ", "ସୁର୍ଯ୍ୟମନ୍ଦିର", "ଧୂଳିଗିରି", "ନନ୍ଦନକାନନ",
    "ଗଣେଶପୂଜା", "ଦୁର୍ଗାପୂଜା", "କାଳୀପୂଜା", "ଲକ୍ଷ୍ମୀପୂଜା", "ସରସ୍ୱତୀପୂଜା",
    "ପ୍ରଭାତୀ", "ମହାପ୍ରଭୁ", "ମାଁତରିଣୀ", "ମାଁସାମଲେଶ୍ୱରୀ", "ମାଁମଣିକେଶ୍ୱରୀ",
    "ମାଁକାନକଦୁର୍ଗା", "ବିମଳା", "ଭାଗବତ", "ଗୀତା", "ରାମାୟଣ", "ମହାଭାରତ",
    "ଗୋପବନ୍ଧୁ", "ବିଜୁପଟ୍ନାୟକ", "ନବୀନପଟ୍ନାୟକ", "ହରିହର", "ବିରାଜା", "ଚନ୍ଦ୍ରଭାଗା",
    "ମହେନ୍ଦ୍ରତନୟ", "ଅଶୋକ", "ଧରିତ୍ରୀ", "ସାମ୍ବ", "ନୀଳାଚଳ", "ପଟିଆ", "ନୟାପଲ୍ଲୀ"]
      self.compound_roots=compound_roots or [

                                            "ଶିକ୍ଷା", "ନୀତି", "ସଂସ୍କୃତି", "କେନ୍ଦ୍ର", "ବିଭାଗ", "ପୁସ୍ତକ", "ମେଳା",
    "ପ୍ରଶାସନ", "ନିୟମ", "ପରୀକ୍ଷା", "ମନ୍ତ୍ରଣାଳୟ", "ପ୍ରଧାନ", "ମନ୍ତ୍ରୀ",
    "ବିଶ୍ୱବିଦ୍ୟାଳୟ", "ସଂସ୍ଥା", "ନିୟୋଜନ", "ବିଭାଗ", "ସମିତି", "ଅଧିକାରୀ",
    "ଚିକିତ୍ସା", "ପ୍ରତିଷ୍ଠାନ", "ଶିଶୁ", "ସେବା", "ଉଦ୍ୟୋଗ", "ଆୟୋଜନ",
    "ତଥ୍ୟ", "ପ୍ରଯୁକ୍ତି", "ଖେଳ", "କ୍ରୀଡା", "ଅଭିଯାନ", "ଅନୁଷ୍ଠାନ",
    "ସାଂସ୍କୃତିକ", "ବିଜ୍ଞାନ", "ତନ୍ତ୍ର", "ଅନୁସନ୍ଧାନ", "ପ୍ରକାଶନ",
    "ବ୍ୟବସ୍ଥା", "ବଜେଟ", "ଉଦ୍‌ଘାଟନ", "ଅଧିବେଶନ", "ବୈଠକ", "ଉପସ୍ଥାପନ",
    "ନୀତି", "ଯୋଜନା", "ପଦକ୍ଷେପ", "ପ୍ରତିବେଦନ", "ସମୀକ୍ଷା", "ସୂଚନା",
    "ସୁରକ୍ଷା", "ନିର୍ବାଚନ", "ପ୍ରଚାର", "ନିୟମାବଳୀ", "ନିୟମକ", "ସଚିବ",
    "ପରିଚାଳନା", "ପାଠ୍ୟକ୍ରମ", "ଅନୁମୋଦନ", "ଶ୍ରମ", "ଉନ୍ନତି", "ଅନୁଶାସନ"
      ]
      self.force_suffix_split=force_suffix_split
      self.enable_fuzzy_split=enable_fuzzy_split

    def normalize_unicode(self, text):
      normalized_text=unicodedata.normalize("NFC", text)
      cleaned = normalized_text.replace('\u200d', '').replace('\u200c', '')
      return cleaned


    def is_known_token(self,token):
      # sentencepiece tokenizer sometimes have "__token" structure in their vocab
      return token in self.vocab or f"__{token}" in self.vocab

    def get_graphemes(self,word):
      return regex.findall(r'\X', word)

    def is_named_entity(self, word):
      for entity in self.named_entities:
          if self.enable_fuzzy_split:
              if word.startswith(entity) or word == entity:
                  return True
          else:
              if word == entity:
                  return True
      return False

    def compound_splitter(self,word):
      graphemes=self.get_graphemes(word)
      for i in range(2,len(graphemes)):
        left="".join(graphemes[:i])
        right="".join(graphemes[i:])

        if left in self.compound_roots and right in self.compound_roots:
          print(f"Compound root split: '{word}' → '{left}' + '{right}'")
          return f"{left} {right}"
      return word

    def split_hyphenated(self, word):
      if "-" in word:
          parts = word.split("-")
          if len(parts) == 2 and all(len(p) > 1 for p in parts):
              print(f"🔨 Hyphen split: '{word}' → '{parts[0]} {parts[1]}'")
              return f"{parts[0]} {parts[1]}"
      return word

    def split_suffix(self, word, depth=0):
        indent = "  " * depth
        graphemes = self.get_graphemes(word)

        for i in range(1, len(graphemes)):
            stem = "".join(graphemes[:i])
            suffix = "".join(graphemes[i:])

            if suffix in self.suffixes and len(stem) > 1:
              # disable unknown token check on an ad hoc basis
                if self.force_suffix_split or  not self.is_known_token(stem) or not self.is_known_token(suffix):

                    print(f"{indent}📍 Split (grapheme-safe): '{word}' → '{stem}' + '{suffix}'")
                    split_stem = self.split_suffix(stem, depth + 1)
                    return f"{split_stem} {suffix}"
                else:
                    print(f"{indent}⚠️ Not splitting: '{word}' → parts unknown")
        print(f"{indent}✅ Final stem: '{word}'")
        return word


    def preprocess_text(self,text):
      normalized=self.normalize_unicode(text)
      words=normalized.split()

      processed=[]
      for word in words:

        if self.is_named_entity(word):
            print(f"Skipping named entity(fuzzy): {word}")
            processed.append(word)
        else:
            word = self.split_hyphenated(word)
            word = self.compound_splitter(word)
            word = self.split_suffix(word)
            processed.append(word)
      return " ".join(processed)

    def visualize_token_split(self, text, return_stats=False):
      print("\n🧪 ORIGINAL INPUT")
      print("📌 Text:", text)

      # --- Raw tokenization ---
      raw_tokens = self.tokenizer.tokenize(text)
      print("\n🧩 RAW TOKENIZATION")
      print(f"🧱 Tokens ({len(raw_tokens)}):", raw_tokens)

      # --- Preprocessing ---
      split_text = self.preprocess_text(text)

      # --- Preprocessed tokenization ---
      pre_tokens = self.tokenizer.tokenize(split_text)
      print("\n🛠️ AFTER PREPROCESSING")
      print("🧼 Cleaned:", split_text)
      print(f"🔤 Tokens ({len(pre_tokens)}):", pre_tokens)

      # --- Comparison ---
      print("\n📊 TOKEN COUNT COMPARISON")
      delta = len(raw_tokens) - len(pre_tokens)
      if delta > 0:
          print(f"✅ Token count reduced by: {delta} tokens")
      elif delta < 0:
          print(f"⚠️ Token count increased by: {-delta} tokens")
      else:
          print("➖ Token count unchanged")

      print("—" * 50)

      if return_stats:
          return {
              "original": text,
              "preprocessed": split_text,
              "raw_tokens": raw_tokens,
              "pre_tokens": pre_tokens,
              "delta": delta
          }

    def encode(self, text, **kwargs):
      preprocessed=self.preprocess_text(text)
      encoded=self.tokenizer.encode(preprocessed, **kwargs)
      return encoded


In [147]:
wrapper=TokenizerWrapper(tokenizer,enable_fuzzy_split=True)
words = [
    "ଶିକ୍ଷାନୀତି ଓ ଶିକ୍ଷା-ବ୍ୟବସ୍ଥା ଉପରେ ଆଲୋଚନା ହେଉଛି ।",
    "ଜଗନ୍ନାଥପୁରୀରେ ଗଣେଶପୂଜା ଓ ଓଡ଼ିଶା-ସଂସ୍କୃତି ପ୍ରଦର୍ଶନୀ ହେଉଛି ।",
     "ପିଲାମାନେକୁ ଶିକ୍ଷାବିଭାଗରୁ ନିୟମଜାରି କରାଯାଇଛି ।",
     "ସେ ବେଳେ-ବେଳେ ଶିକ୍ଷା-ନୀତି ଉପରେ-ଆଲୋଚନା କରନ୍ତି ।",
     "ତ୍ରିଭୁବନେଶ୍ୱରରେ ଅନେକ ଶିକ୍ଷାନୀତିକ ବିଚାର ହେଉଛି ।",
     "ସେ ଆପଣଙ୍କୁ ବିଶ୍ୱାସ କରନ୍ତି ।"

]

wrapper.visualize_token_split(words[2])


🧪 ORIGINAL INPUT
📌 Text: ପିଲାମାନେକୁ ଶିକ୍ଷାବିଭାଗରୁ ନିୟମଜାରି କରାଯାଇଛି ।

🧩 RAW TOKENIZATION
🧱 Tokens (123): ['▁', '<0xE0>', '<0xAC>', '<0xAA>', '<0xE0>', '<0xAC>', '<0xBF>', '<0xE0>', '<0xAC>', '<0xB2>', '<0xE0>', '<0xAC>', '<0xBE>', '<0xE0>', '<0xAC>', '<0xAE>', '<0xE0>', '<0xAC>', '<0xBE>', '<0xE0>', '<0xAC>', '<0xA8>', '<0xE0>', '<0xAD>', '<0x87>', '<0xE0>', '<0xAC>', '<0x95>', '<0xE0>', '<0xAD>', '<0x81>', '▁', '<0xE0>', '<0xAC>', '<0xB6>', '<0xE0>', '<0xAC>', '<0xBF>', '<0xE0>', '<0xAC>', '<0x95>', '<0xE0>', '<0xAD>', '<0x8D>', '<0xE0>', '<0xAC>', '<0xB7>', '<0xE0>', '<0xAC>', '<0xBE>', '<0xE0>', '<0xAC>', '<0xAC>', '<0xE0>', '<0xAC>', '<0xBF>', '<0xE0>', '<0xAC>', '<0xAD>', '<0xE0>', '<0xAC>', '<0xBE>', '<0xE0>', '<0xAC>', '<0x97>', '<0xE0>', '<0xAC>', '<0xB0>', '<0xE0>', '<0xAD>', '<0x81>', '▁', '<0xE0>', '<0xAC>', '<0xA8>', '<0xE0>', '<0xAC>', '<0xBF>', '<0xE0>', '<0xAD>', '<0x9F>', '<0xE0>', '<0xAC>', '<0xAE>', '<0xE0>', '<0xAC>', '<0x9C>', '<0xE0>', '<0xAC>', '<0xBE>', '<0xE0>